<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item nDCG</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_ii = knn.ItemItem(neighbours,center=center,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    results = rla.compute(all_recs, test_data)
    return results["ndcg"].mean()

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[17.27549244664848, 'True', 0.10035811943173294]                                                                       
,                                                                                                                      
[89.37909195815375, 'True', 0.0918961923734985]                                                                        
,                                                                                                                      
[93.64674322109137, 'False', 0.09174044353410835]                                                                      
,                                                                                                                      
[93.98954106833627, 'False', 0.09470985843958091]                                                                      
,                                                                                                                      
[30.172655875199588, 'False', 0.10109683

[25.303754452975404, 'False', 0.10170318054169512]                                                                     
,                                                                                                                      
[99.64051680062454, 'False', 0.08970102151583995]                                                                      
,                                                                                                                      
[65.09199339799031, 'False', 0.09791088928477885]                                                                      
,                                                                                                                      
[46.63723651604767, 'True', 0.09654547643389003]                                                                       
,                                                                                                                      
[8.202804614490251, 'False', 0.100845105

[1.2081553653580777, 'False', 0.07767745808264667]                                                                     
,                                                                                                                      
[17.50207640442695, 'False', 0.09814290654784573]                                                                      
,                                                                                                                      
[4.579711346946856, 'False', 0.09681147808522562]                                                                      
,                                                                                                                      
[31.37396777727536, 'False', 0.1014112467725791]                                                                       
,                                                                                                                      
[37.93837062124407, 'False', 0.096195019

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','nDCG']
metric

,neighbours,center,nDCG
0,17.275492,True,0.100358
1,89.379092,True,0.091896
2,93.646743,False,0.091740
3,93.989541,False,0.094710
4,30.172656,False,0.101097
...,...,...,...
95,16.330034,False,0.100734
96,25.731816,False,0.096846
97,9.715717,True,0.100342
98,22.036911,False,0.101719


In [6]:
metric.sort_values(by=['nDCG'],ascending=False).head(10)

,neighbours,center,nDCG
56,15.414311,False,0.104638
27,13.263236,False,0.103708
73,14.812376,False,0.102889
24,11.308100,False,0.102888
78,13.540409,True,0.102268
50,10.879571,True,0.102121
93,12.498410,False,0.101926
98,22.036911,False,0.101719
34,25.303754,False,0.101703
8,12.492267,False,0.101665


In [7]:
metric.to_csv('metric_nDCG.csv', index=False)